In [1]:
from Kramer.database.MongoDB_CRUD import get_all_courses_sync
from rich.console import Console
import chromadb
import dspy
import os

In [2]:
print("Getting courses...")
courses = get_all_courses_sync()
trainset = []
for course in courses:
    try:
        topic = course.course_title
        intro_video = course.sections[0].entries[0].transcript
        trainset.append(dspy.Example(topic=topic, intro_video=intro_video))
    except:
        pass
print(f"Got {len(trainset)} courses.")
trainset = [x.with_inputs("topic", "intro_video") for x in trainset]
print(len(trainset))

Getting courses...
Got 11008 courses.
11008


In [4]:
# Set up the console for pretty printing
console = Console(width=100)
# Set up the OpenAI API key and instantiate the GPT-4o model
api_key = os.getenv("OPENAI_API_KEY")
# lm = dspy.LM("ollama_chat/llama3.1", api_base="http://localhost:11434", api_key="")
# lm = dspy.LM("ollama_chat/llama3.1", api_base="http://localhost:11434", api_key="")
lm = dspy.LM("openai/gpt-4o-mini", api_key=api_key)
dspy.configure(lm=lm)

In [5]:
class WriteVideoTranscript(dspy.Signature):
    """Write a video transcript for the intro video to a course on a given topic."""

    topic: str = dspy.InputField(desc="the topic of the course")
    intro_video: str = dspy.OutputField(
        desc="markdown-formatted transcript of an intro video of roughly 250 words"
    )

In [6]:
def get_similarity_score(original_text: str, generated_text: str):
    """Get the similarity score between two texts."""
    if not isinstance(original_text, str) or not isinstance(generated_text, str):
        return 0.0  # Return a default poor score instead of None

    client = chromadb.Client()

    # Clean up previous collection if exists
    try:
        client.delete_collection("similarity_test")
    except:
        pass

    collection = client.create_collection("similarity_test")

    try:
        collection.add(
            documents=[original_text, generated_text], ids=["original", "generated"]
        )

        results = collection.query(
            query_texts=[original_text], n_results=2, include=["distances"]
        )

        similarity_score = results["distances"][0][1]
        return float(similarity_score)
    except Exception as e:
        print(f"Error in similarity calculation: {e}")
        return 0.0  # Return a default poor score instead of None
    finally:
        # Clean up
        try:
            client.delete_collection("similarity_test")
        except:
            pass

In [7]:
write_video_transcript = dspy.ChainOfThought(WriteVideoTranscript)

In [8]:
# Plot the data
# import pandas as pd
# import matplotlib.pyplot as plt
# scores = [s for s in scores if s is not None]
# s=pd.Series(scores)
# mean = sum(scores) / len(scores)
# print(mean)
# # Histogram
# s.hist(bins=30)  # you can adjust number of bins as needed
# plt.title('Distribution of Similarity Scores')
# plt.xlabel('Similarity Score')
# plt.ylabel('Frequency')
# plt.show()
# # plt.scatter(s.index, s.values, alpha = .1)
# s.describe()
# s.to_csv("scores_500.csv", index = False)
"""
count    498.000000
mean       1.719879
std        0.163160
min        0.548691
25%        1.622713
50%        1.728114
75%        1.837510
max        2.147124
"""

'\ncount    498.000000\nmean       1.719879\nstd        0.163160\nmin        0.548691\n25%        1.622713\n50%        1.728114\n75%        1.837510\nmax        2.147124\n'

In [9]:
def metric(example, pred, trace=None):
    topic, intro_video, transcript = (
        example.topic,
        example.intro_video,
        pred.intro_video,
    )

    similarity_score = get_similarity_score(intro_video, transcript)
    try:
        verdict = similarity_score >= 1.9
        return verdict
    except:
        return None

In [10]:
scores = []
n = 200
for index, x in enumerate(trainset[:n]):
    print(f"Evaluating {index+1} out of {n}")
    pred = write_video_transcript(**x.inputs())
    score = metric(x, pred)
    scores.append(score)

Evaluating 1 out of 200
Evaluating 2 out of 200
Evaluating 3 out of 200
Evaluating 4 out of 200
Evaluating 5 out of 200
Evaluating 6 out of 200
Evaluating 7 out of 200
Evaluating 8 out of 200
Evaluating 9 out of 200
Evaluating 10 out of 200
Evaluating 11 out of 200
Evaluating 12 out of 200
Evaluating 13 out of 200
Evaluating 14 out of 200
Evaluating 15 out of 200
Evaluating 16 out of 200
Evaluating 17 out of 200
Evaluating 18 out of 200
Evaluating 19 out of 200
Evaluating 20 out of 200
Evaluating 21 out of 200
Evaluating 22 out of 200
Evaluating 23 out of 200
Evaluating 24 out of 200
Evaluating 25 out of 200
Evaluating 26 out of 200
Evaluating 27 out of 200
Evaluating 28 out of 200
Evaluating 29 out of 200
Evaluating 30 out of 200
Evaluating 31 out of 200
Evaluating 32 out of 200
Evaluating 33 out of 200
Evaluating 34 out of 200
Evaluating 35 out of 200
Evaluating 36 out of 200
Evaluating 37 out of 200
Evaluating 38 out of 200
Evaluating 39 out of 200
Evaluating 40 out of 200
Evaluatin

In [11]:
import matplotlib.pyplot as plt
import numpy as np

In [12]:
def visualize_bool_list(bool_list):
    # Calculate proportions
    total = len(bool_list)
    true_count = sum(bool_list)
    proportions = [true_count / total, (total - true_count) / total]

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(12, 2))

    # Create horizontal bar
    ax.barh(0, proportions[0], color="#2ecc71", label=f"True ({true_count})")
    ax.barh(
        0,
        proportions[1],
        left=proportions[0],
        color="#e74c3c",
        label=f"False ({total-true_count})",
    )

    # Customize appearance
    ax.set_yticks([])  # Remove y-axis ticks
    ax.set_xticks([])  # Remove x-axis ticks
    ax.set_xlim(0, 1)  # Set x-axis limits

    # Remove all spines
    for spine in ax.spines.values():
        spine.set_visible(False)

    # Add percentage labels
    ax.text(
        proportions[0] / 2,
        0,
        f"TRUE ({true_count}/{total})",
        ha="center",
        va="center",
        color="white",
        fontsize=12,
        fontweight="bold",
    )
    ax.text(
        proportions[0] + proportions[1] / 2,
        0,
        f"FALSE ({total-true_count}/{total})",
        ha="center",
        va="center",
        color="white",
        fontsize=12,
        fontweight="bold",
    )

    plt.tight_layout()
    plt.show()

Example usage:
scores = [s for s in scores if s is not None]
bool_list = scores
visualize_bool_list(bool_list)


In [13]:
teleprompter = dspy.MIPROv2(
    metric=metric,
    num_threads=24,
    verbose=True,
    max_bootstrapped_demos=4,
    max_labeled_demos=16,
)
# Train the model
optimized_program = teleprompter.compile(write_video_transcript, trainset=trainset[:100])

Projected Language Model (LM) Calls

Based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Prompt Generation: 10 data summarizer calls + 10 * 1 lm calls in program + (2) lm calls in program-aware proposer = 22 prompt model calls
- Program Evaluation: 25 examples in minibatch * 30 batches + 80 examples in val set * 4 full evals = 1070 LM Program calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of program calls * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expecta

2025/01/11 18:03:14 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/11 18:03:14 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/11 18:03:14 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=10 sets of demonstrations...


Bootstrapping set 1/10
Bootstrapping set 2/10
Bootstrapping set 3/10


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:01<00:00, 19.26it/s]


Bootstrapped 0 full traces after 19 examples for up to 1 rounds, amounting to 20 attempts.
Bootstrapping set 4/10


...

Average Metric: 0.00 / 0 (0%):  12%|██████████▋                                                                              | 3/25 [00:00<00:05,  4.40it/s]

2025/01/11 18:06:43 ERROR dspy.utils.parallelizer: Error processing item Example({'topic': 'Sketch Essential Training: The Basics', 'intro_video': " - Welcome to the LinkedIn course,  \n Sketch essential training ,the basics.  \n At the end of this video,  \n you'll have an overview of what the course is about,  \n as well as an expectation of what you're going  \n to get out of it.  \n This course is for anyone who wants to learn how to create  \n digital products using Sketch,  \n by the end of it,  \n you should understand enough about the tool, to create,  \n test and share digital design assets.  \n We're going to go through the various features of Sketch  \n by creating the elements that we need for mobile app,  \n for a bicycle shop.  \n We'll cover things like understanding the tool  \n as well as how to use it to accomplish various design tasks.  \n I'm Chimmy Kalu,  \n and I create lovable products.  \n Why don't you watch this course  \n and learn how to use Sketch with me. 

Error in similarity calculation: Collection d2fe2bd6-3a5a-42f4-8680-6e2252f7388d does not exist.
Average Metric: 0.00 / 19 (0.0%):  92%|██████████████████████████████████████████████████████████████████████████████▏      | 23/25 [00:05<00:00,  8.97it/s]

2025/01/11 18:06:48 WARNING dspy.utils.parallelizer: Received SIGINT. Cancelling execution.
[W 2025-01-11 18:06:48,629] Trial 29 failed with parameters: {'0_predictor_instruction': 1, '0_predictor_demos': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/bianders/Brian_Code/.Magnus/lib/python3.12/site-packages/dspy/utils/parallelizer.py", line 176, in _execute_multi_thread
    for future in as_completed(futures):
  File "/usr/lib/python3.12/concurrent/futures/_base.py", line 243, in as_completed
    waiter.event.wait(wait_timeout)
  File "/usr/lib/python3.12/threading.py", line 655, in wait
    signaled = self._cond.wait(timeout)
               ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/threading.py", line 355, in wait
    waiter.acquire()
  File "/home/bianders/Brian_Code/.Magnus/lib/python3.12/site-packages/dspy/utils/parallelizer.py", line 132, in interrupt_handler
    default_handler(sig, frame)
KeyboardInterrupt

Dur

Average Metric: 0.00 / 19 (0.0%):  96%|█████████████████████████████████████████████████████████████████████████████████▌   | 24/25 [00:21<00:00,  8.97it/s]

In [ ]:
# Save the optimized program to disk
optimizeda_program.save("intro_video.json")
optimized_program.save("intro_video.pkl")